In [143]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
%load_ext rpy2.ipython

import sys
import os

nb_dir = os.path.split(os.getcwd())[0]

sys.path.append(nb_dir)
import pandas as pd
import numpy as np
import scipy as sp
from GELMMnet import GELMMnet, kinship
print(nb_dir)

from sklearn.metrics import mean_squared_error

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
The rpy2.ipython extension is already loaded. To reload it, use:
  %reload_ext rpy2.ipython
/Users/bs224/Dropbox/PostDoc/projects/GELMMnet


In [16]:
from GELMMnet import kinship, GELMMnet, normalized_laplacian, transform_kernel_to_distance
import scipy as SP

geno_filename = os.path.join(nb_dir,'./test/genotypes.csv')
X = SP.genfromtxt(geno_filename)
y = SP.genfromtxt(os.path.join(nb_dir,"test/phenotypes.csv"))
K = kinship(X)
P = np.identity(X.shape[1])
n_s,n_v = X.shape
n_train = 150
n_test = n_s - n_train


# split into training and testing
train_idx = SP.random.permutation(SP.arange(n_s))
test_idx = train_idx[n_train:]
train_idx = train_idx[:n_train]

# X[train_idx],K[train_idx][:,train_idx],y[train_idx]

print("P Dim:",P.shape)
print("X Dim", X.shape)
glm = GELMMnet(y[train_idx], X[train_idx], K[train_idx][:,train_idx])
glm.train(P, l1=1, l2=0.5, max_iter=1000, eps=1.0e-5)

y_pred = glm.predict(X[test_idx])
msr_regreg = mean_squared_error(y[test_idx],y_pred)
w_regreg = glm.w
print("W",w_regreg.sum())
print("MSR:",msr_regreg)

P Dim: (1000, 1000)
X Dim (176, 1000)
W 0.611532483529
MSR: 0.711151836402


In [247]:
%%R -o x,y,out
library(glmnet)
library(selectiveInference)
set.seed(43)
n = 50
p = 10
sigma = 1
x = matrix(rnorm(n*p),n,p)
x=scale(x,TRUE,TRUE)
beta = c(3,2,rep(0,p-2))
y = x%*%beta + sigma*rnorm(n)
# first run glmnet
gfit = glmnet(x,y,standardize=FALSE)
# extract coef for a given lambda; note the 1/n factor!
# (and we don't save the intercept term) fixedLassoInf 7 
lambda = .8
beta = coef(gfit, s=lambda/n, exact=TRUE)[-1]
print(beta)
# compute fixed lambda p-values and selection intervals
out = fixedLassoInf(x,y,beta,lambda,sigma=sigma)
out





 [1]  2.90117931  2.09140871  0.10536154  0.07723765 -0.17150755 -0.13135784
 [7]  0.13166391  0.02265503 -0.09731043  0.00000000

Call:
fixedLassoInf(x = x, y = y, beta = beta, lambda = lambda, sigma = sigma)

Standard deviation of noise (specified or estimated) sigma = 1.000

Testing results at lambda = 0.800, with alpha = 0.100

 Var   Coef Z-score P-value LowConfPt UpConfPt LowTailArea UpTailArea
   1  2.912  18.897   0.000     2.655    3.196       0.048      0.049
   2  2.123  13.803   0.000     1.856    2.377       0.049      0.049
   3  0.141   0.817   0.495    -0.728    0.404       0.049      0.049
   4  0.109   0.699   0.578    -0.854    0.337       0.050      0.049
   5 -0.206  -1.240   0.257    -0.486    0.310       0.048      0.049
   6 -0.145  -0.968   0.358    -0.395    0.392       0.048      0.049
   7  0.141   0.909   0.374    -0.432    0.477       0.050      0.049
   8  0.045   0.304   0.865    -2.872    0.165       0.050      0.050
   9 -0.113  -0.722   0.510    -0.35

In [252]:
%%R
library(glmnet)
library(selectiveInference)

# Returns Prob(Z>z | Z in [a,b]), where mean can be a vector

tnorm.surv <- function(z, mean, sd, a, b, bits=NULL) {
  z = max(min(z,b),a)
  
  # Check silly boundary cases
  p = numeric(length(mean))
  p[mean==-Inf] = 0
  p[mean==Inf] = 1
  
  # Try the multi precision floating point calculation first
  o = is.finite(mean)
  mm = mean[o]
  pp = mpfr.tnorm.surv(z,mm,sd,a,b,bits)
  # If there are any NAs, then settle for an approximation
  oo = is.na(pp)
  if (any(oo)) pp[oo] = bryc.tnorm.surv(z,mm[oo],sd,a,b)
    
  p[o] = pp
  #print(paste("z",z,"mean",mean,"sd",sd,"a",a,"b",b,"pval",p))
  return(p)
}

# Returns Prob(Z>z | Z in [a,b]), where mean cane be a vector, using
# multi precision floating point calculations thanks to the Rmpfr package

mpfr.tnorm.surv <- function(z, mean=0, sd=1, a, b, bits=NULL) {
  # If bits is not NULL, then we are supposed to be using Rmpf
  # (note that this was fail if Rmpfr is not installed; but
  # by the time this function is being executed, this should
  # have been properly checked at a higher level; and if Rmpfr
  # is not installed, bits would have been previously set to NULL)
  #if (!is.null(bits)) {
  #  z = Rmpfr::mpfr((z-mean)/sd, precBits=bits)
  #  a = Rmpfr::mpfr((a-mean)/sd, precBits=bits)
  #  b = Rmpfr::mpfr((b-mean)/sd, precBits=bits)
  #  return(as.numeric((Rmpfr::pnorm(b)-Rmpfr::pnorm(z))/
  #                    (Rmpfr::pnorm(b)-Rmpfr::pnorm(a))))
  #}
  
  # Else, just use standard floating point calculations
  z = (z-mean)/sd
  a = (a-mean)/sd
  b = (b-mean)/sd
  return((pnorm(b)-pnorm(z))/(pnorm(b)-pnorm(a)))
}

# Returns Prob(Z>z | Z in [a,b]), where mean can be a vector, based on
# A UNIFORM APPROXIMATION TO THE RIGHT NORMAL TAIL INTEGRAL, W Bryc
# Applied Mathematics and Computation
# Volume 127, Issues 23, 15 April 2002, Pages 365--374
# https://math.uc.edu/~brycw/preprint/z-tail/z-tail.pdf

bryc.tnorm.surv <- function(z, mean=0, sd=1, a, b) {
  z = (z-mean)/sd
  a = (a-mean)/sd
  b = (b-mean)/sd
  n = length(mean)

  term1 = exp(z*z)
  o = a > -Inf
  term1[o] = ff(a[o])*exp(-(a[o]^2-z[o]^2)/2)
  term2 = rep(0,n)
  oo = b < Inf
  term2[oo] = ff(b[oo])*exp(-(b[oo]^2-z[oo]^2)/2)
  p = (ff(z)-term2)/(term1-term2)

  # Sometimes the approximation can give wacky p-values,
  # outside of [0,1] ..
  #p[p<0 | p>1] = NA
  old = p
  p = pmin(1,pmax(0,p))

  return(p)
}

ff <- function(z) {
  return((z^2+5.575192695*z+12.7743632)/
         (z^3*sqrt(2*pi)+14.38718147*z*z+31.53531977*z+2*12.77436324))
}

# Return Prob(Z>z | Z in [a,b]), where mean can be a vector, based on
# Riemann approximation tricks, by Max G'Sell

gsell.tnorm.surv <- function(z, mean=0, sd=1, a, b) {
  return(max.approx.frac(a/sd,b/sd,z/sd,mean/sd))
}

    
pinv <- function(A, tol=.Machine$double.eps) {
  e = eigen(A)
  v = Re(e$vec)
  d = Re(e$val)
  d[d > tol] = 1/d[d > tol]
  d[d < tol] = 0
  if (length(d)==1) return(v*d*v)
  else return(v %*% diag(d) %*% t(v))
}

      
poly.pval <- function(y, G, u, v, sigma, bits=NULL) {
  z = sum(v*y)
  vv = sum(v^2)
  sd = sigma*sqrt(vv)
  
  rho = G %*% v / vv
 
  vec = (u - G %*% y + rho*z) / rho
  
  vlo = suppressWarnings(max(vec[rho>0]))
  vup = suppressWarnings(min(vec[rho<0]))

  pv = tnorm.surv(z,0,sd,vlo,vup,bits)
  print(pv)
  return(list(pv=pv,vlo=vlo,vup=vup))
}
      
      
poly.int <- function(y, G, u, v, sigma, alpha, gridrange=c(-100,100),
                     gridpts=100, griddepth=2, flip=FALSE, bits=NULL) {
  
  z = sum(v*y)
  vv = sum(v^2)
  sd = sigma*sqrt(vv)
  
  rho = G %*% v / vv
  vec = (u - G %*% y + rho*z) / rho
  vlo = suppressWarnings(max(vec[rho>0]))
  vup = suppressWarnings(min(vec[rho<0]))
  
  xg = seq(gridrange[1]*sd,gridrange[2]*sd,length=gridpts) 
  
  fun = function(x) { tnorm.surv(z,x,sd,vlo,vup,bits) }

  int = grid.search(xg,fun,alpha/2,1-alpha/2,gridpts,griddepth)
 
  tailarea = c(fun(int[1]),1-fun(int[2]))

  if (flip) {
    int = -int[2:1]
    tailarea = tailarea[2:1]
  }
  
  return(list(int=int,tailarea=tailarea))
}

##############################

# Assuming that grid is in sorted order from smallest to largest,
# and vals are monotonically increasing function values over the
# grid, returns the grid end points such that the corresponding
# vals are approximately equal to {val1, val2}

grid.search <- function(grid, fun, val1, val2, gridpts=100, griddepth=2) {
  n = length(grid)
  vals = fun(grid)
  
  print(vals)
    
  ii = which(vals >= val1)
  jj = which(vals <= val2)
  if (length(ii)==0) return(c(grid[n],Inf))   # All vals < val1
  if (length(jj)==0) return(c(-Inf,grid[1]))  # All vals > val2
  # RJT: the above logic is correct ... but for simplicity, instead,
  # we could just return c(-Inf,Inf) 
 
  i1 = min(ii); i2 = max(jj)
  print(paste("i2 ", i2, val2))
  if (i1==1) lo = -Inf
  else lo = grid.bsearch(grid[i1-1],grid[i1],fun,val1,gridpts,
         griddepth-1,below=TRUE)
  if (i2==n) hi = Inf
  else hi = grid.bsearch(grid[i2],grid[i2+1],fun,val2,gridpts,
         griddepth-1,below=FALSE)
 
  return(c(lo,hi))
}

# Repeated bin search to find the point x in the interval [left, right]
# that satisfies f(x) approx equal to val. If below=TRUE, then we seek
# x such that the above holds and f(x) <= val; else we seek f(x) >= val.

grid.bsearch <- function(left, right, fun, val, gridpts=100, griddepth=1, below=TRUE) {
  n = gridpts
  depth = 1
  print(paste("left ",left, " right ",right, " val ", val))

  while (depth <= griddepth) {
    grid = seq(left,right,length=n)
    vals = fun(grid)
    
    if (below) {
      ii = which(vals >= val)
      print(paste(below," ii "))
      print(ii)
      if (length(ii)==0){
          #print(paste(below," ii empty ", grid[n]))
          return(grid[n])
      }    # All vals < val (shouldn't happen)
      if ((i0=min(ii))==1){
          #print(paste(below," i0==1 ", grid[1]))
          return(grid[1])
      }  # All vals > val (shouldn't happen)
      left = grid[i0-1]
      right = grid[i0]
    }
    
    else {
      ii = which(vals <= val)
      print(paste(below," ii "))
      print(ii)
      if (length(ii)==0) {
          #print(paste(below," ii empty ", grid[1]))
          return(grid[1])   # All vals > val (shouldn't happen)
        }
      if ((i0=max(ii))==n){ 
          #print(paste(below," i0==n ", grid[n]))
          return(grid[n]) # All vals < val (shouldn't happen)
        }
      left = grid[i0]
      right = grid[i0+1]
    }

    depth = depth+1
  }
  #print(paste(below, " bound ", ifelse(below, left, right)))
  return(ifelse(below, left, right))
}
      
set.seed(43)
n = 50
p = 10
sigma = 1
x = matrix(rnorm(n*p),n,p)
x=scale(x,TRUE,TRUE)
beta = c(3,2,rep(0,p-2))
y = x%*%beta + sigma*rnorm(n)
gfit = glmnet(x,y,standardize=FALSE)
# extract coef for a given lambda; note the 1/n factor!
# (and we don't save the intercept term) fixedLassoInf 7 
lambda = .8
beta = coef(gfit, s=lambda/n, exact=TRUE)[-1]
      
w = beta
a = w != 0


xa = x[,a,drop=F]
xac = x[,!a,drop=F]
xai = pinv(crossprod(xa))
xap = xai %*% t(xa)
za = sign(beta[a])
dz = diag(za)

G = -rbind(
   #   1/lambda * t(xac) %*% P,
   # -1/lambda * t(xac) %*% P,
    -dz %*% xap
      )
u = -c(
   #   1 - t(xac) %*% t(xap) %*% za,
   #   1 + t(xac) %*% t(xap) %*% za,
    -lambda * dz %*% xai %*% za)

 
M = xap
j = 2      
vj = M[j,]
mj = sqrt(sum(vj^2))
vj = vj / mj        # Standardize (divide by norm of vj)
sign_j = sign(sum(vj*y))
print(sign_j)
vj = sign_j * vj
vj
r = poly.pval(y,G,u,vj,1.)
vmat = vj * mj * sign_j
gridrange=c(-100,100)
alpha=0.05
          
a = poly.int(y,G,u,vj,sigma,alpha,gridrange=gridrange,
      flip=(sign_j==-1),bits=NULL)
ci = a$int * mj # Unstandardize (mult by norm of vj)
tailarea = a$tailarea
sd = sigma*sqrt(sum(vmat^2))
print(paste("Ci ",ci," mj ", mj, " sd ",sd))
      
round(ci,3)

[1] 1
[1] 7.364081e-16
  [1] 5.932295e-137 1.663816e-134 4.666386e-132 1.308724e-129 3.670350e-127
  [6] 1.029338e-124 2.886686e-122 8.095269e-120 2.270140e-117 6.365943e-115
 [11] 1.785095e-112 5.005495e-110 1.403523e-107 3.935297e-105 1.103367e-102
 [16] 3.093472e-100  8.672703e-98  2.431328e-95  6.815722e-93  1.910547e-90
 [21]  5.355247e-88  1.500979e-85  4.206697e-83  1.178898e-80  3.303521e-78
 [26]  9.256357e-76  2.593353e-73  7.265027e-71  2.034989e-68  5.699414e-66
 [31]  1.596009e-63  4.468579e-61  1.250903e-58  3.500964e-56  9.795997e-54
 [36]  2.740260e-51  7.663006e-49  2.142145e-46  5.985691e-44  1.671715e-41
 [41]  4.666066e-39  1.301450e-36  3.626802e-34  1.009601e-31  2.806628e-29
 [46]  7.788677e-27  2.156516e-24  5.952676e-22  1.636168e-19  4.469754e-17
 [51]  1.209780e-14  0.000000e+00  0.000000e+00  2.065933e-07  4.486003e-05
 [56]  6.585602e-03  2.551857e-01  9.112382e-01  9.996221e-01  1.000000e+00
 [61]  1.000000e+00  1.000000e+00  1.000000e+00  1.000000e+00  1.

In [253]:
from GELMMnet import GELMMnet

g = GELMMnet(y, x, np.identity(x.shape[0]))
loss, w = g.train(np.identity(x.shape[1]), lmm=False, l1=0.8, l2=0, eps=1e-8)
g.post_selection_analysis(lmm=False,alpha=0.1,compute_intervals=True)

beta: [ 2.90096224  2.09152418  0.10563911  0.07744745 -0.17163314 -0.13123873
  0.13172712  0.02272241 -0.09728269 -0.        ]


/Users/bs224/Dropbox/PostDoc/projects/GELMMnet/GELMMnet/GELMMnet.py:320: RuntimeWarning: invalid value encountered in double_scalars
  p = (norm.cdf(bb) - norm.cdf(zz)) / (norm.cdf(bb) - norm.cdf(aa))
/Users/bs224/Dropbox/PostDoc/projects/GELMMnet/GELMMnet/GELMMnet.py:328: RuntimeWarning: overflow encountered in exp
  # https://math.uc.edu/~brycw/preprint/z-tail/z-tail.pdf
/Users/bs224/Dropbox/PostDoc/projects/GELMMnet/GELMMnet/GELMMnet.py:330: RuntimeWarning: overflow encountered in exp
  if aa > -np.inf:
/Users/bs224/Dropbox/PostDoc/projects/GELMMnet/GELMMnet/GELMMnet.py:333: RuntimeWarning: overflow encountered in exp
  if bb < np.inf:
/Users/bs224/Dropbox/PostDoc/projects/GELMMnet/GELMMnet/GELMMnet.py:334: RuntimeWarning: invalid value encountered in double_scalars
  term2 = ff(bb) * np.exp(-(bb**2 - zz**2) / 2)
/Users/bs224/Dropbox/PostDoc/projects/GELMMnet/GELMMnet/GELMMnet.py:365: RuntimeWarning: invalid value encountered in greater_equal
  
/Users/bs224/Dropbox/PostDoc/projects

,pval,coef0,beta,Zscore,lower_confidence,upper_confidence,lower_trunc,upper_trunc,sd
variable,,,,,,,,,
0.0,0.000000,2.911576,2.900962,343.817563,0.846837,inf,0.000000,0.000000,0.008468
1.0,0.000000,2.122611,2.091524,251.143377,0.845179,inf,0.000000,0.000000,0.008452
2.0,0.000000,0.141078,0.105639,14.856207,0.125280,0.156866,0.048089,0.048205,0.009496
3.0,0.000000,0.109437,0.077447,12.723478,0.095218,0.123652,0.049147,0.049206,0.008601
4.0,0.000000,-0.205650,-0.171633,-22.562943,-0.220679,-0.190548,0.049587,0.048769,0.009115
5.0,0.000000,-0.144647,-0.131239,-17.608818,-0.158321,-0.130999,0.047984,0.048310,0.008214
6.0,0.000000,0.141072,0.131727,16.537498,0.126986,0.155186,0.049343,0.049011,0.008530
7.0,0.000005,0.045156,0.022722,5.530623,0.031073,0.058730,0.049478,0.048200,0.008165
8.0,0.000000,-0.112833,-0.097283,-13.143687,-0.127091,-0.098712,0.048364,0.049999,0.008585
